# Managing products
Here we will outline the basic steps to create and manage new raster `Product`s and `Image`s.

In [ ]:
from descarteslabs.catalog import (
    Image,
    DataType,
    Product,
    Resolution,
    ResolutionUnit,
    SpectralBand,
    properties as p,
)

## Creating and updating a product
Before uploading images to the catalog, you need to create a product and declare its bands. The only required attributes for a product are a unique id and a name:

In [ ]:
# Creating a unique ID to avoid conflicts:
from uuid import uuid4

pid = f"guide-example-product-{uuid4()}"
name = "Example product"

In [ ]:
product = Product()
product.id = pid
product.name = name
product.tags = ["examples"]
product.save()
product.id

Every object has a read-only `created` attribute with the timestamp from when it was first saved:

In [ ]:
product.created

`save()` saves the product to the catalog in the cloud. Note that you get to choose an id for your product but it must be unique within your organization (you get an exception if it’s not). This code example is assuming the user is in the “descarteslabs” organization. The id is prefixed with the organization id on save to enforce global uniqueness and uniqueness within an organization. If you are not part of an organization the prefix will be your unique user id. You can find this unique user id on your IAM page if you click on your name in the upper right.

There are a few more attributes that you can set (see the [`Product`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product) API reference). You can update the product to define the timespan that it covers. This is as simple as assigning attributes and then saving again:

In [ ]:
product.start_datetime = "2012-01-01"
product.end_datetime = "2015-01-01"
product.save()
product.start_datetime

A read-only `modified` attribute exists on all objects and is updated on every save.

In [ ]:
product.modified

Note that all timestamp attributes are represented as `datetime` instances in UTC. You may assign strings to timestamp attributes if they can be reasonably parsed as timestamps. Once the object is saved the attributes will appear as parsed `datetime` instances. If a timestamp has no explicit timezone, it’s assumed to be in UTC.

## Creating bands
Before adding any images to a product you must create bands that declare the structure of the data shared among all images in a product.

In [ ]:
band = SpectralBand(name="blue", product=product)
band.data_type = DataType.UINT16
band.data_range = (0, 10000)
band.display_range = (0, 4000)
band.resolution = Resolution(unit=ResolutionUnit.METERS, value=60)
band.band_index = 0
band.save()
band.id

A band is uniquely identified by its name and product. The full id of the band is composed of the product id and the name.

The band defines where its data is found in the files attached to images in the product: In this example, `band_index = 0` indicates that `blue` is the first band in the image file, and that first band is expected to be represented by unsigned 16-bit integers (`DataType.UINT16`).

This band is specifically a `SpectralBand`, with pixel values representing measurements somewhere in the visible/NIR/SWIR electro-optical wavelength spectrum, so you can also set additional attributes to locate it on the spectrum:

In [ ]:
# These values are in nanometers (nm)
band.wavelength_nm_min = 452
band.wavelength_nm_max = 512
band.save()

See the [`Bands`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html#bands) Documentation page for more information about other band types not covered in this example.

# Managing images
Apart from searching and discovering data available to you, catalog enables you to upload new images of your own.

## Creating images
There are two general mechanisms of creating images in the catalog. Upload is the primary mechanism for creating images, either by uploading supported image files types such as GeoTIFF or JPEG, or by uploading image data in the form of a numpy ndarray. The other mechanism is to create “remote” image entries in the catalog without supplying the actual image data.

### Uploading image files
If your data already exists on disk as an image file, usually a GeoTIFF or JPEG file, you can upload it directly.

In the following examples we will upload data to the single band we created representing the blue light spectrum. 

Now you create a new image and use `image.upload()` to upload imagery to the new product. This returns a ImageUpload. Images are uploaded and processed asynchronously, so they are not available in the catalog immediately. With `upload.wait_for_completion()` we wait until the upload is completely finished.

In [ ]:
# Set any attributes that should be set on the uploaded images
image = Image(product=product, name="image1")
image.acquired = "2012-01-02"
image.cloud_fraction = 0.1
# Do the upload
image_path = "data/blue.tif"
upload = image.upload(image_path)
upload.wait_for_completion()
upload.status

## Access Control
By default only the creator of a product can read and modify it as well as read and modify the bands and images in it. To share access to a product with others you can modify its access control lists (ACLs):

In [ ]:
product.readers = ["org:descarteslabs"]
product.writers = ["email:jane.doe@descarteslabs.com", "email:john.daly@gmail.com"]
product.save()

This gives read access to the whole “descarteslabs” organization. All users in that organization can now find the product. This also gives write access to two specific users identified by email. These two users can now update the product and add new images to it. For further information on access control please see the [Sharing Resources](https://docs.descarteslabs.com/guides/sharing.html) guide.

### Transfer ownership
Transfering ownership of a product to a new user requires cooperation from both the previous owner and the new owner and is a two-step effort. The first step is for the previous owner to add the new owner to the product:

Just a reminder that you cannot use the `email:` variant as an owner. You will have to request the user id from the new owner and use that instead. (You can find your user id in the profile drop-down on iam.descarteslabs.com).

The second step is for the new owner to remove the previous owner:

## Deleting bands and products
Any catalog objects (Products, Bands, and Images) can be deleted using the `delete` method:

In [ ]:
product.delete()

#### *NOTE:*
A product can _only be deleted if it doesn’t have any associated bands or images_. Because the product we created still has images and bands we must first delete those related objects. 

There is a convenience method to delete all bands and images in a product. Be careful as this may delete a lot of data and can’t be undone!

In [ ]:
status = product.delete_related_objects()

This kicks off a job that deletes bands and images in the background. You can wait for this to complete and then delete the product:

In [ ]:
if status:
    status.wait_for_completion()
product.delete()

If you have run this notebook multiple times, you can easily delete all Products with the `examples` tag as follows:

In [ ]:
import descarteslabs as dl

user_hash = dl.auth.Auth().namespace
for prod in (
    Product.search()
    .filter(p.tags == "examples")
    .filter(p.owners == f"user:{user_hash}")
):
    print(f"Deleting {prod.id}")
    status = prod.delete_related_objects()
    if status:
        status.wait_for_completion()
    prod.delete()